In [1]:
import os

In [2]:
from keras.preprocessing import image

In [ ]:
import cv2

In [ ]:
categories = ['with_mask', 'without_mask']

In [ ]:
data = []
for category in categories:
    path = os.path.join('dataset', category)
   
    label = categories.index(category)

    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224,224))
        data.append([img, label])

In [ ]:
data

In [ ]:
len(data)

In [ ]:
import random
random.shuffle(data)

In [ ]:
X =[]
Y =[]
for features, label in data:
    X.append(features)
    Y.append(label)

In [ ]:
import numpy as np
X = np.array(X)
Y = np.array(Y)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X = X /255

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state= 10)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16()

In [ ]:
from keras import Sequential

In [ ]:
model = Sequential()

In [ ]:
for layer in vgg.layers[:-1]:
    model.add(layer)

model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable=False

In [ ]:
from keras.layers import Dense

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs= 5, validation_data=(X_test, Y_test))

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    return y_pred

In [ ]:
def draw_label(img, text, position, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    end_x = position[0] + text_size[0][0] + 2
    end_y = position[1] + text_size[0][1] - 2
    cv2.rectangle(img, position, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, position,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

In [ ]:
while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to capture image from camera.")
        break
    else:
        image = cv2.resize(frame, (224, 224))
        y_pred = detect_face_mask(image)

        if y_pred == 1:
            label, color = "Mask", (0, 255, 0)
        else:
            label, color = "No Mask", (0, 0, 255)

        draw_label(frame, label, (100, 100), color)
        cv2.imshow("window", frame)

    if cv2.waitKey(10) & 0xff == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()
